
# ECE795 Advanced Big Data Analytics Homework 3 (Due 2/24)

Set up Pyspark Environment.

Tips for Colab:

1. You will be disconnected if you are idle for more than 90 minutes and will be mandatorily disconnected after 12 hour connection. 

2. Once you got disconnected, you need to execute the codes from the beginning to setup the environment again.

3. For the purpose of homework, it should be sufficient since each problem should not take more than 5 minutes to generate the results.

4. To facilitate the use of Colab, you can use "MainMenu - Runtime - Run all” to run all the cells in the notebook. So you do not have to click each cell to setup the environment.

In [1]:
!pip install pyspark
!pip install -q findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 64.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=67e4d789607441476f041577786744a58954f3edbd6dd13dcc129415546b420c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Congrats! Your Colab is ready to run Pyspark.

# Read input text file to RDD 

Download the input data from using the following command.

In [2]:
!wget https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_3/Gutenberg.txt

--2022-02-23 01:56:38--  https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_3/Gutenberg.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3536418 (3.4M) [text/plain]
Saving to: ‘Gutenberg.txt’

Gutenberg.txt       100%[===================>]   3.37M  --.-KB/s    in 0.05s   

2022-02-23 01:56:38 (73.8 MB/s) - ‘Gutenberg.txt’ saved [3536418/3536418]



Now that we have input data, we can start to do the homework. 

## Question 1 (10 points): The following RDD, 'words', is obtained from Homework 2, Question 3. It contains all the words in the text file, 'Gutenberg.txt', after pre-processing. Using this RDD, please create another RDD called 'frequencies' that contains the term frequencies of all words.

### Example output from 'frequencies.take(10)':
```
[('project', 95),
 ('ten', 208),
 ('use', 35),
 ('anyone', 5),
 ('anywhere', 6),
 ('united', 20),
 ('world', 39),
 ('whatsoever', 2),
 ('may', 84),
 ('give', 155)]
```

In [3]:
#Question_1:
from pyspark import SparkConf, SparkContext
import nltk
nltk.download('stopwords')
nltk.download('words')
stop_words = set(nltk.corpus.stopwords.words('english'))
english_words = set(nltk.corpus.words.words())

def removeNonAlpabet(s):
    return ''.join([i.lower() for i in s if i.isalpha() or i==' ']).lstrip().rstrip()

sc = SparkContext.getOrCreate()
sentences = sc.textFile('Gutenberg.txt').map(removeNonAlpabet).filter(lambda x: x!='')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [14]:
# Question 1
#Fill out here

words = sentences.flatMap(lambda line: line.split())
frequencies = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)  # .sortBy(lambda x: -x[1])
# test
frequencies.take(10)

[('project', 95),
 ('gutenberg', 32),
 ('ebook', 13),
 ('of', 6820),
 ('engelsch', 24),
 ('ten', 208),
 ('bruggencate', 2),
 ('this', 494),
 ('is', 2621),
 ('use', 35)]

## Question 2 (10 points): For the 'frequencies' RDD in Question 1, normalize the values of term frequencies so that for every feature, the minimum value of that feature gets transformed into a 0, the maximum value gets transformed into a 1, and every other value gets converted into a decimal between 0 and 1 (also referred to as Min-Max normalization).

The min-max normalized term frequency of the $n^{th}$ word can be calculated as:

## $\left\Vert f_n \right\Vert_{minmax} = \frac{f_n-\min{f}}{\max{f}-\min{f}}$

where $f_n$ and $\left\Vert f_n \right\Vert_{minmax}$ are the term frequencies of the $n^{th}$ word before and after min-max normalization; $\min{f}$,$\max{f}$ are the minimum and maximum frequencies before normalization.

In [16]:
#Question_2
#Fill out here

import numpy as np
words = sentences.flatMap(lambda line: line.split())
frequencies = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)  # .sortBy(lambda x: -x[1])  [('the', 1.0),('subst', 0.9920370638482698)]
frequencies_list = np.array(frequencies.map(lambda x: x[1]).collect())
f_max, f_min = frequencies_list.max(), frequencies_list.min()
f_maxmin = frequencies.map(lambda x: (x[0], ((x[1] - f_min)) / (f_max - f_min)))
# f_maxmin.collect()
# test
f_maxmin.take(10)

[('project', 0.013609381786593312),
 ('gutenberg', 0.004488200376429709),
 ('ebook', 0.00173736788765021),
 ('of', 0.9872593021572318),
 ('engelsch', 0.0033299551179962355),
 ('ten', 0.02996959606196612),
 ('bruggencate', 0.00014478065730418415),
 ('this', 0.07137686405096279),
 ('is', 0.3793253221369625),
 ('use', 0.0049225423483422615)]

## Question 3 (10 points): For the 'frequencies' RDD in Question 1, normalize the values of term frequencies through Z-Score Normalization.

The Z-Score Normalization term frequency of the $n^{th}$ word can be calculated as:

## $\left\Vert f_n \right\Vert_z = \frac{f_n - \mu}{\sigma}$

where $\mu$ is the mean value of the feature and $\sigma$ is the standard deviation of the feature. If a value is exactly equal to the mean of all the values of the feature, it will be normalized to 0. If it is below the mean, it will be a negative number, and if it is above the mean it will be a positive number. The size of those negative and positive numbers is determined by the standard deviation of the original feature. If the unnormalized data had a large standard deviation, the normalized values will be closer to 0.

In [15]:
#Question_3
#Fill out here

import numpy as np
words = sentences.flatMap(lambda line: line.split())
frequencies = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y) # .sortBy(lambda x: -x[1])
frequencies_list = np.array(frequencies.map(lambda x: x[1]).collect())
f_mu, f_sigma = frequencies_list.mean(), frequencies_list.std()
f_zscore = frequencies.map(lambda x: (x[0], (x[1] - f_mu) / f_sigma))
# f_zscore.collect()
# test
f_zscore.take(10)

[('project', 1.3788788810367412),
 ('gutenberg', 0.4212583524560268),
 ('ebook', 0.13245216129676377),
 ('of', 103.60107022556537),
 ('engelsch', 0.2996557456521266),
 ('ten', 3.096515702141832),
 ('bruggencate', -0.034751423058599046),
 ('this', 7.443808895381264),
 ('is', 39.77490197936824),
 ('use', 0.46685933000748936)]